In [1]:
import os
import pandas as pd
import numpy as np
import textwrap
import re

## Read and Pre-processing Data

##### Reading files

In [2]:
path = os.getcwd() # get the current directory where the script is placed
path_f = f'{path}\\input_source_file.xlsx'
path_r = f'{path}\\input_ref_file.xlsx'

In [3]:
full_data = pd.read_excel(path_f,
                          sheet_name="DataTable",
                          dtype={'Ship To ID':str,
                          'Order Date':'datetime64[ns]'})  #read table autochem extract for bryan

In [4]:
# creating dataframe for each sheets in files
ptlist = pd.read_excel(path_r, sheet_name="allparts", na_values=['None'],dtype={'MTPN':str,'Associated Site Part to Increment':str})  #read partlists table
ctryrep = pd.read_excel(path_r, sheet_name="CountryNameRep",keep_default_na=False)  #read country name representation table and keep auto-detecting na function off (NA:north america)
region = pd.read_excel(path_r,sheet_name="Region",na_values=["N/A", "#N/A"],keep_default_na=False)  #read region table
id = pd.read_excel(path_r,sheet_name='SiteNames',dtype={'Ship-To Party':str}).rename({'Ship-To Party':'Ship To ID','Site Name':'Customer'},axis=1)

##### Data Pre-Processing

In [5]:
# data cleaning
id['Ship To ID'] = id['Ship To ID'].apply(lambda x: x.lstrip('0'))  #force column type to eliminate leading zeros then to str
full_data['Ship To ID'] = full_data['Ship To ID'].apply(lambda x: x.lstrip('0'))  #strip leading zeros

#select needed columns from full data and rename
sub_data = full_data[[
    'Geo Country', 'Sales Org', 'Model ID', 'Model', 'SVC Mat ID',
    'Material ID', 'Material', 'Product Type', 'Order Number',
    'Sold To Customer ID', 'Sold To Customer', 'Ship To ID',
    'Ship To Customer', 'Ship To Country', 'Industry Level 1 - Ship to',
    'Industry Level 2 - Ship to', 'Order Date', 'Month', 'Quarter',
    'List Price', 'Net Sales', 'Quantity']].copy().rename({'Material ID': 'MTPN'},axis=1)

#split qtr col to two called quarter&yr_ordered
sub_data[['Quarter', 'Yr_ordered']] = sub_data.Quarter.str.split(' ', expand=True)
sub_data['Quarter'] = sub_data['Quarter'].replace('Q4','Q0')  #replace Q4 with Q0
sub_data['Yr_ordered'] = sub_data['Yr_ordered'].astype(int)  #se the yr col type to int

#save year as int type and assign to a new column Year Alloc and assign Q0 to the next year
sub_data['Yr_alloc'] = sub_data['Yr_ordered'].mask(sub_data['Quarter'] == 'Q0',sub_data['Yr_ordered'] + 1)

#create a new country column with fixed country names
sub_data['Country'] = sub_data['Ship To Country'].replace(ctryrep['Bad Name'].tolist(), ctryrep['Rename'].tolist())
sub_data = sub_data.merge(region, on='Country',how='left')  #add region column
#sub_data[['Region', 'Country']].value_counts() # check data

sub_data['unitLP'] = sub_data['List Price'] / sub_data['Quantity']  #create a unitLP column
sub_data['discount'] = sub_data['Net Sales'] / sub_data['List Price']  #create a discount column
#sub_data.info()

##### HW and SL List

In [6]:
#create a sitelist of category in Site lisc
hwlist = ptlist[ptlist['Category'] == 'Hardware'].reset_index(drop=True)
#hwlist = hwlist.rename({'MTPN':'SVC Mat ID'},axis=1) #rename material ID to MTPN for inner join
sitelist = ptlist[ptlist['Category'] == 'Site'].reset_index(drop=True)

##### Customer Name and expiration List

In [7]:
#read current SL cus datafile and rename the column
slcus_exp = pd.read_excel(path_r, sheet_name='SL_exp', dtype = {'SL_exp_dt': 'datetime64[ns]'})\
              .rename({'Name': 'Customer','SL Expiration Date': 'SL_exp_dt'}, axis=1)  
for row, col in slcus_exp.iterrows(): # ** made change here so the contract length is showing as numerical and future changes will be automated
    for x in col:
        slcus_exp.loc[row, "contract_start"] = col['SL_exp_dt'] - pd.DateOffset(years= int(col['Contract Length'].split()[0]))
slcus_exp_id = pd.merge(slcus_exp,id[['Ship To ID','Customer']],on='Customer',how='inner') # match and merge SiteName and SL_exp sheets 

#slcus_exp.head()
#slcus_exp_id.head()

#pd.merge(slcus_exp,id,on='Customer',how='outer',indicator=True).query('_merge == "right_only"') # checking SiteNames and SL_exp sheets where doesnt match (expired customers)

-----
## SL Data Processing
* Get all the SL orders

##### SL data and Customer Contract List and Get aggregated active Customers SL orders

In [8]:
#inner join two dataframe by MTPN to get all the SL orders
sl_orders = pd.merge(sub_data, sitelist, on='MTPN', how='inner')
sl_orders = sl_orders.drop(sl_orders[sl_orders['Quantity'] == 0].index) #data check --> subset where quantity = 0 and drop rows
sl_orders.isna().sum() #check NA rows/ get na order numbers
#site_data.info()

# merge with ship to id sheet to match and get the active SL orders
# define a function and call to generate orders that include expiration date info
def get_fullsl(all_sl_orders):
    order_exp = pd.merge(all_sl_orders, slcus_exp_id, on='Ship To ID', how='inner') # merge to get the matching records on both sheets
    con = ((order_exp['contract_start'] - pd.DateOffset(months=3)) < order_exp['Month']) & ((order_exp['Month'] <= order_exp['SL_exp_dt'].dt.to_period('M'))) # set a condition
    order_exp = order_exp[con == True].reset_index(drop=True) # select condition in the merged sl order sheet: order_exp
    
    #generate aggregated SL orders that ordered within their current term
    orders = order_exp[[
        'Customer', 'MTPN', 'unitLP', 'SPG', 'SubCat', 'Country', 'Region',
        'Order Date', 'Contract Length', 'discount', 'Net Sales', 'SL_exp_dt',
        'Quantity'
    ]].rename({'Quantity': 'SL_quant'}, axis=1) # select needed columns and rename Quantity
    uniq_MTPN = order_exp.groupby(['Customer'])['MTPN'].unique().explode() #get the unique MTPN list of SL purchased by each customer
    # merge to get unique MTPN list ordered by each customer
    uniq_MTPN = pd.merge(orders,uniq_MTPN, on=['Customer', 'MTPN'], how='inner')
    # group and aggregate to get mean net sales/dis for each subCat and maximum purchased quanity of each device under SubCat
    agg_orders = uniq_MTPN.groupby(['Customer', 'MTPN', 'SPG', 'SubCat', 'Contract Length',
                        'Country','Region', 'SL_exp_dt']).agg({'SL_quant': 'max','unitLP': 'mean',
                        'Net Sales': 'mean','discount': 'mean'}).sort_values(by=['Customer', 'MTPN']).reset_index()
    return order_exp, agg_orders
sheets = get_fullsl(sl_orders)

sl_orders_exp = sheets[0] # subset 1st sheet and call it 
agg_sl_orders = sheets[1] # subset 2nd sheet and call it 

#sl_orders_exp.head()
#agg_sl_orders.info()

----
## Hardware Data
* Get all the HW orders

In [9]:
# keep only the hardwares that are associated with Site License
hw_orders = pd.merge(sub_data, hwlist, on='MTPN', how='inner')\
              .rename({'Associated Site Part to Increment': 'SL_MTPN', 'Quantity': 'HW_quant'}, axis=1)\
              .dropna(subset=['SL_MTPN'])  # merge, rename, dropna of 'SL_MTPN' column
#hw_orders.info()

# define and call function to get hw orders
def get_hworder(cus_exp_data):
    active_hw = pd.merge(cus_exp_data, hw_orders, on='Ship To ID', how='inner')  # get hw data based on customer contract yr
    active_hw = active_hw[[
        'Customer','Country', 'Description', 'Contract Length', 'SL_exp_dt', 'contract_start',
        'SubCat', 'Order Number', 'License Desc', 'SL_MTPN', 'Order Date','HW_quant']] #subset these columns
    order_con = (active_hw['contract_start'] < active_hw['Order Date']) & (active_hw['Order Date'] <= (active_hw['SL_exp_dt']))# save only the records within 3 yr contract period
    active_hw = active_hw[order_con] #subset rows where condition is true
    # group by these columns and aggregate by the sum of hardware quantity and rename
    active_hw = active_hw.groupby(['Customer', 'SL_MTPN', 'SubCat',
                            'Country', 'Order Date','Order Number']).agg({'HW_quant': 'sum'}).reset_index()\
                         .rename({'Order Date': 'hw_order_dt'}, axis=1) 
    active_hw['hw_order_yr'] = active_hw['hw_order_dt'].dt.year #create a new column of ordered year 
    return active_hw
hw_orders_exp = get_hworder(slcus_exp_id)
#hw_orders_exp.head()

----
## Data Check

##### Check possible Missing IDs

In [10]:
#orders - check possible missing ship to IDs
def get_missing_ids(orders):
    missing_ids = pd.merge(id, orders, on='Ship To ID', how='outer', indicator=True)
    missing_ids = missing_ids.query('_merge ==  "right_only"')[['Model','Sold To Customer','Ship To ID',
                                                 'Ship To Customer','Country','Region',
                                                 'Order Number','Order Date','SubCat']].value_counts().reset_index().drop_duplicates(subset=['Ship To ID','Country'])
    return missing_ids
inactive_sl_customers = get_missing_ids(sl_orders)
missing_hw_ids = get_missing_ids(hw_orders)


##### Write to excel file

In [11]:
with pd.ExcelWriter('output_sl_hw_details.xlsx',engine='xlsxwriter') as writer:
    # Each call to to_excel creates a new sheet in our excel file output.xlsx
    sl_orders.to_excel(writer,sheet_name='all_SL Orders',index=False)
    agg_sl_orders.to_excel(writer,sheet_name='agg SL Orders',index=False)
    hw_orders.to_excel(writer,sheet_name='all_HW_orders',index=False)
    hw_orders_exp.to_excel(writer, sheet_name='HW orders assoc SL', index=False)
    inactive_sl_customers.to_excel(writer, sheet_name='SL-other IDs', index=False)
    missing_hw_ids.to_excel(writer, sheet_name='HW-other IDs', index=False)

----
## Forcast
To get totalt quantity of SL for calculating the New Quote

In [12]:
#define and call function to merge to get the quantity change 
def sl_hw_merge(x, y):
    sl_col = ['Customer', 'MTPN', 'Country', 'SubCat']
    hw_col = ['Customer', 'SL_MTPN', 'Country', 'SubCat']
    qt_change = pd.merge(x, y, how='left', left_on=sl_col, right_on=hw_col).fillna(0)
    qt_change['SL_exp_yr'] = qt_change['SL_exp_dt'].dt.year #creating a column of exp yr for comparsion 
    
    #aggregate Hardware quantity
    revenue_forcast = qt_change.groupby([
    'Customer', 'Region', 'Country', 'SL_exp_dt', 'SL_exp_yr',
    'Contract Length', 'SPG', 'SubCat', 'MTPN', 'SL_MTPN', 'unitLP',
    'discount', 'Net Sales', 'SL_quant']).agg({'HW_quant': 'sum'}).reset_index() #to aggregate the HW quantity by other others
    revenue_forcast['total_qt'] = revenue_forcast['HW_quant'] + revenue_forcast['SL_quant']  #calculate the new quantity
    
    # create a separate file of cross table of customer names and expiration years
    cus = pd.DataFrame(revenue_forcast['Customer'].unique().tolist(), columns=['Customer']) #get the unique list and name it 'Customer'
    yr = pd.DataFrame(revenue_forcast['SL_exp_yr'].unique().tolist(), columns=['Year']) #get the unique list of SL exp years and name it 'Year'
    cusyr_cross = cus.merge(yr, how='cross').sort_values(by=['Customer', 'Year']).reset_index(drop=True) # merge to get the cross list of customer names and year
    return qt_change, revenue_forcast, cusyr_cross
sheets = sl_hw_merge(agg_sl_orders,hw_orders_exp)

# write to excel file
qt_change = sheets[0].to_excel('output_qt_change.xlsx') 
revenue_forcast = sheets[1].to_excel('output_revenue_forecast.xlsx')
cusyr_cross = sheets[2].to_excel('output_CusYr_cross.xlsx')